In [1]:
import sqlite3
import csv
import os
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('encounters.db')
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Export each table to a separate CSV file
for table in tables:
    table_name = table[0]
    csv_file_path = f'{table_name}.csv'
    
    # Execute a query to select all data from the table
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()
    
    # Write data to the CSV file
    # Certain characters break if not UTF-8 enocoded, might be the korean?
    with open(csv_file_path, 'w',encoding="utf-8", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([description[0] for description in cursor.description])
        csvwriter.writerows(rows)

    print(f"Exported table '{table_name}' to '{csv_file_path}'")

conn.close()

Exported table 'encounter' to 'encounter.csv'
Exported table 'entity' to 'entity.csv'


In [2]:
def remove_junk_names(df):
    ''' not sure why this occurs but to remove non player names that get in there run this over the death df'''
    df = df[df['name'].str.contains('\d') == False]
    return df

entity_df = pd.read_csv('entity.csv')
entity_df = remove_junk_names(entity_df)
encounter_df = pd.read_csv('encounter.csv')

In [3]:
# Grab the boss in question and only look at the players
one_boss_all_encounters = entity_df[entity_df['name']=='Thaemine the Lightqueller']
one_boss_all_encounters = entity_df[entity_df['encounter_id'].isin(one_boss_all_encounters['encounter_id'])]
players = one_boss_all_encounters[one_boss_all_encounters['entity_type'] == 'PLAYER']

# The combat details are under damage_stats so to merge them back in we reset the index then concat them together.
# there is a lot of worthless columns here so maybe remove those later idc
players.reset_index(drop=True,inplace=True)
from pandas import json_normalize
damage_stats = json_normalize(players['damage_stats'].apply(eval))
players_full_detail = pd.concat([players,damage_stats], axis = 1)
# Get death information

# we just care about deaths for this
player_data_clean = players_full_detail[['name','encounter_id','deathTime']]

# If someone doesn't die the deathtime = 0 so to make it so it doesn't look like they die first I am setting the death time for them to 9999999999999
player_data_clean.loc[player_data_clean['deathTime'] == 0, 'deathTime'] = 9999999999999

# Use when you want to see all the columns
# players.columns.tolist()

In [4]:
# filter out the people who live until restart
encounter_death_info = encounter_df[['id','last_combat_packet','fight_start','duration']]
encounter_death_info = encounter_death_info.rename(columns = {'id':'encounter_id',
                                        'last_combat_packet' :'last_combat_packet',
                                        'fight_start' : 'fight_start',
                                        'duration' : 'duration'})

# Remove deaths from restarts and kill pulls
removed_deaths = player_data_clean.merge(encounter_death_info,on='encounter_id',how='left')
removed_deaths = removed_deaths[removed_deaths['deathTime'] != removed_deaths['last_combat_packet']]
player_data_clean = removed_deaths[['name','encounter_id','deathTime']]

In [5]:
def first_death_grouper(df):
    ''' Only keep the first group of players who died.'''
    # only keep the first death group
    min_deathTime = df.groupby(['encounter_id'])['deathTime'].transform('min')
    # Filter the DataFrame to include only rows where the deathTime is the smallest in the encounter_id
    first_death_group = df[df['deathTime'] == min_deathTime].groupby(['encounter_id', 'name','deathTime']).size().reset_index(name='count').drop('count',axis=1)
    # We could have added this at a prior point but I just thought of it so it is being added here.
    # This connects the encounter times, so we can remove the final deathtimes with the final combat death. Seeing how the last group of people aren't really causing the wipe we want to exclude them
    first_death_group
    return first_death_group
    
first_death_group = first_death_grouper(player_data_clean)

In [7]:
def nth_death_grouper(full_df,first_deaths_df):
    '''grab the next layer of deaths past the first group
    You can run this on the result df in a loop to drill down deeper into groups'''
    drilled_down_deaths = full_df.merge(first_deaths_df, on='deathTime', how='left', indicator=True)
    result_df = drilled_down_deaths[drilled_down_deaths['_merge'] == 'left_only'].drop(columns='_merge')
    # fix names
    result_df = result_df[['name_x','encounter_id_x','deathTime']].rename(columns={'name_x': 'name',
                                                                                     'encounter_id_x' : 'encounter_id',
                                                                                     'deathTime': 'deathTime'})
    return result_df

second_group_deaths_df = nth_death_grouper(player_data_clean,first_death_group)

In [8]:
first_death_group = remove_junk_names(first_death_group)
# first_death_group.groupby(['name']).count().reset_index()

In [9]:
second_group_deaths_df = remove_junk_names(second_group_deaths_df)
second_group_deaths_df.groupby(['name']).count().reset_index()

,name,encounter_id,deathTime
0,Babythigh,145,145
1,Chickenwings,110,110
2,Djgogo,22,22
3,Eiskres,85,85
4,Imsocheerios,138,138
5,Jabazat,134,134
6,Lauracharis,114,114
7,Poesía,127,127
8,Zousha,119,119
9,Zoushayaya,20,20


In [10]:
# add in a custom metric for prog and 1st reclear
prog_data = player_data_clean[player_data_clean['encounter_id'] < 510]
reclear_data = player_data_clean[player_data_clean['encounter_id'] >= 510]

# prog data
# Group deaths and remove junk names
def parse_first_second_groups(df):
    first_group_df = remove_junk_names(first_death_grouper(df))
    second_group_df = remove_junk_names(nth_death_grouper(df,first_group_df))
    
    first_group_df = first_group_df.groupby(['name']).count().reset_index()
    second_group_df = second_group_df.groupby(['name']).count().reset_index()
    
    merged_group = first_group_df.merge(second_group_df, on='name', how='left').drop(['deathTime_x','deathTime_y'],axis=1)
    merged_group = merged_group.rename(columns={'name':'name',
                               'encounter_id_x' : 'first_death_group_count',
                                'encounter_id_y' : 'second_death_group_count'
                               })
    return merged_group

In [11]:
parse_first_second_groups(reclear_data)

,name,first_death_group_count,second_death_group_count
0,Babythigh,1,22
1,Djgogo,1,22
2,Eiskres,12,11
3,Imsocheerios,1,22
4,Jabazat,2,20
5,Poesía,8,15
6,Zoushayaya,2,20
